# Video Segmentation using pyAudioAnalysis
This is a note notebook implementing the audio segmentation function of 
pyAudioAnalysis to cut pauses and silence out of long videos.

In [4]:
from pyAudioAnalysis import audioSegmentation as aS
from pathlib import Path
import soundfile as sf
import ipywidgets as widgets
from IPython.display import display
from ipyfilechooser import FileChooser

In [6]:
input_path = Path("./input")
fc = FileChooser(str(input_path))
display(fc)

FileChooser(path='input', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='Fal…

In [7]:
video_path = input_path / fc.selected_filename
output_path = Path("./outputs/") / video_path.stem
if output_path.exists():
    !rm -rf {output_path}
output_path.mkdir(parents=True, exist_ok=True)

First we must extract the audio from the input video. We do this using `ffmpeg` from the command line.

In [8]:
audio_path = output_path / (video_path.stem + ".wav")
!ffmpeg -i {video_path} {audio_path} -y

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [9]:
print("Choose window size in ms")
window_slider = widgets.FloatSlider(min=0.0, max=500.0, step=5, value=250)
display(window_slider)
    
print("Choose step size in ms")
step_slider = widgets.FloatSlider(min=0.0, max=5.0, step=0.1, value=2.9)
display(step_slider)

Choose window size in ms


FloatSlider(value=250.0, max=500.0, step=5.0)

Choose step size in ms


FloatSlider(value=2.9, max=5.0)

In [10]:
window_size = window_slider.value / 1000
step_size = step_slider.value / 1000

data, fs = sf.read(audio_path)
segment_limits = aS.silence_removal(data, fs, window_size, step_size)

In [11]:
print(f"Process audio into {len(segment_limits)} segments.")
total_removed = segment_limits[-1][1] - sum([y - x for x,y in segment_limits])
print(f"This removed a total of {total_removed:.2f} seconds from the original video.")

Process audio into 174 segments.
This removed a total of 155.23 seconds from the original video.
[[0.40599999999999997, 5.341799999999999], [6.8816999999999995, 7.2703], [8.282399999999999, 11.692799999999998], [12.174199999999999, 14.001199999999999], [14.340499999999999, 16.7475], [17.0578, 20.4363], [22.7012, 23.1739], [24.8269, 27.912499999999998], [28.651999999999997, 30.2992], [30.913999999999998, 31.238799999999998], [31.5491, 33.155699999999996], [33.727, 35.641], [36.5458, 37.2273], [37.6971, 38.2191], [40.246199999999995, 40.7827], [41.1539, 41.667199999999994], [43.9118, 44.152499999999996], [44.7354, 46.507299999999994], [46.6262, 47.310599999999994], [47.586099999999995, 48.1545], [48.580799999999996, 48.980999999999995], [49.070899999999995, 49.630599999999994], [49.9728, 50.9762], [52.8844, 54.1575], [56.2223, 56.57899999999999], [58.481399999999994, 62.5791], [62.9648, 64.47569999999999], [65.1108, 66.265], [67.07119999999999, 67.6454], [68.07169999999999, 70.3453], [70

In [26]:
import math
def s_to_hms(seconds, as_string=False):
    hours = math.floor(seconds / 3600)
    minutes = math.floor(seconds % 3600 / 60)
    s = seconds % 60
    if not as_string:
        return hours, minutes, s
    else:
        seconds_string = f"{s:.2f}" if s >= 10 else f"0{s:.2f}"
        return f"{hours:02}:{minutes:02}:{seconds_string}"

In [27]:
import subprocess
from tqdm import tqdm

for i, (start, end) in enumerate(tqdm(segment_limits, "Creating segments")):
    
    command = f"ffmpeg -i {video_path} -ss {s_to_hms(start, as_string=True)} -to {s_to_hms(end, as_string=True)} -c:v libx264 -c:a copy {output_path}/segment_{i}.mp4 -y"
    print(command)
    !{command};

Creating segments:   0%|          | 0/174 [00:00<?, ?it/s]

ffmpeg -i input/signale.mp4 -ss 00:00:00.41 -to 00:00:05.34 -c:v libx264 -c:a copy outputs/signale/segment_0.mp4 -y
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis 

Creating segments:   1%|          | 1/174 [00:00<02:40,  1.08it/s]

ffmpeg -i input/signale.mp4 -ss 00:00:06.88 -to 00:00:07.27 -c:v libx264 -c:a copy outputs/signale/segment_1.mp4 -y
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis 

Creating segments:   1%|          | 2/174 [00:01<02:12,  1.30it/s]

...

Creating segments:  99%|█████████▉| 172/174 [09:05<00:11,  5.63s/it]

ffmpeg -i input/signale.mp4 -ss 00:06:07.17 -to 00:06:10.56 -c:v libx264 -c:a copy outputs/signale/segment_172.mp4 -y
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbi

Creating segments:  99%|█████████▉| 173/174 [09:11<00:05,  5.77s/it]

ffmpeg -i input/signale.mp4 -ss 00:06:11.03 -to 00:06:12.87 -c:v libx264 -c:a copy outputs/signale/segment_173.mp4 -y
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbi

Creating segments: 100%|██████████| 174/174 [09:17<00:00,  3.20s/it]


In [28]:
segment_list = output_path / "segment_list.txt"
!rm {segment_list}
with open(segment_list, 'w+') as f:
    f.writelines([f"file 'segment_{i}.mp4'\n" for i in range(len(segment_limits))])
!ffmpeg -f concat -safe 0 -i {segment_list} -c copy {output_path}/full.mp4 -y
#!ffmpeg -f concat -safe 0 -i {segment_list} -c copy ./full.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib